In [1]:
# Implementation of OT
from sage.rings.finite_rings.integer_mod import square_root_mod_prime

import time
seed = int(round(time.time() * 10))

class RabinOTParams:
    
    def __init__(self, modbits):
        set_random_seed(987654433)
        current_randstate().set_seed_gp()
        # use gp.random() to get prng random number
        self.p = next_prime(2^modbits + gp.random())
        print("p : ", self.p)
        self.q = next_prime(2^modbits + gp.random())
        print("q : ", self.q)
        if self.p == self.q:
            print("Primes are same, please run again")
        self.n = self.p*self.q
        self.phi_n = (self.p-1)*(self.q-1)
        # Lifting is necessary here, in order for inverse_mod to work correctly
        self.e = mod(gp.random(), self.phi_n).lift()
        while gcd(self.e, self.phi_n)!=1:
            self.e = mod(gp.random(), self.phi_n).lift()
        self.d = inverse_mod(self.e, self.phi_n)
        

class RabinOTSender:
    
    def __init__(self, p, q, n, phi_n, e, d, message):
        self.p = p
        self.q = q
        self.n = n
        self.phi_n = phi_n
        self.e = e
        self.d = d
        self.m = message
        print("The actual message of sender is : ", self.m)
        self.c = self.m.powermod(self.e, self.n)
        # Decryption for debugging
        #print(self.c)
        #self.c = self.c.powermod(self.d, self.n)
        #print(self.c)
        
    def findroot(self, xsq):
        #self.xsq = xsq
        x_p = square_root_mod_prime(mod(xsq, self.p))
        x_q = square_root_mod_prime(mod(xsq, self.q))
        x = mod(x_p, self.p).crt(mod(x_q, self.q))
        return x
        

class RabinOTReceiver:
    
    def __init__(self, n, e, c, caseseed):
        self.n = n
        self.e = e
        self.c = c
        set_random_seed(caseseed)
        # use 98765193 seed to get a y not equal to x as a root of xsq
        current_randstate().set_seed_gp()
        # Lifting required to continue treating x as an integer
        self.x = mod(gp.random(), self.n).lift()
        #print(self.x)
        self.xsq = self.x.powermod(2, self.n)
        
    def revelation(self, y):
        if self.x == y or (self.n - self.x) == y:
            print("Receiver could NOT find what the message was!")
        else:
            y = y.lift()
            diff_roots = abs(y-self.x)
            pprime = gcd(diff_roots, self.n)
            qprime = self.n/pprime
            print("The receiver can factorize n now with (p, q) as : " + "( " + str(pprime) + "," + str(qprime) + " )")
            phi_n = ( pprime-1)*(qprime- 1)
            d = inverse_mod(self.e, phi_n)
            m = self.c.powermod(d, self.n)
            print("The reciever CAN now see the message as : " + str(m))
            print("Reciever was ABLE to find the message")
            
        

class RabinOT:
    
    def __init__(self, modbits, message, case):
        ot = RabinOTParams(modbits)
        ot_sender = RabinOTSender(ot.p, ot.q, ot.n, ot.phi_n, ot.e, ot.d, message)
        if case == 1:
            ot_recv = RabinOTReceiver(ot.n, ot.e, ot_sender.c, 98765194) #y and x are same
        elif case == 2:
            ot_recv = RabinOTReceiver(ot.n, ot.e, ot_sender.c, 98765191) #y and -x are same
        elif case == 3:
            ot_recv = RabinOTReceiver(ot.n, ot.e, ot_sender.c, 98765193) #retreive value of m from a non trivial root
        y = ot_sender.findroot(ot_recv.xsq) # y is one of the 4 square roots of xsq mod n
        ot_recv.revelation(y)
        print("Protocol Complete.")
        
        
modbits = 12
message = 562
print("-----------case 1------------")
case = 1
rabinot = RabinOT(modbits, message, case)
print("-----------case 2------------")
case = 2
rabinot = RabinOT(modbits, message, case)
print("-----------case 3------------")
case = 3
rabinot = RabinOT(modbits, message, case)    


-----------case 1------------
('p : ', 1510763857)
('q : ', 1843463927)
('The actual message of sender is : ', 562)
Receiver could NOT find what the message was!
Protocol Complete.
-----------case 2------------
('p : ', 1510763857)
('q : ', 1843463927)
('The actual message of sender is : ', 562)
Receiver could NOT find what the message was!
Protocol Complete.
-----------case 3------------
('p : ', 1510763857)
('q : ', 1843463927)
('The actual message of sender is : ', 562)
The receiver can factorize n now with (p, q) as : ( 1843463927,1510763857 )
The reciever CAN now see the message as : 562
Reciever was ABLE to find the message
Protocol Complete.
